In [1]:
import PIL
import torch
import glob as gb
import numpy as np
from PIL import Image
import os
import random
import torch.nn as nn
import pandas as pd 

from numpy import dot
from numpy.linalg import norm
import matplotlib.pyplot as plt
import random

from facenet_pytorch import MTCNN, InceptionResnetV1
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn

os.sys.path.append('../')
os.sys.path.append('../models/')

from utils.loss import cos_sim

random.seed(1)
np.random.seed(1)

In [20]:
# Testing upsampling via SR
class SuperResolveDL:
    def __init__(self, model, size, channel, resize=True):
        self.model = model
        self.size = size
        self.channel = channel
        self.resize = resize
        
    def __call__(self, path=''):
        if self.channel == 1:
            img = Image.open(path).convert('YCbCr').resize((self.size,self.size),resample=Image.BICUBIC)
            upsample, cb, cr = img.split()
            if not self.resize:
                upsample, _, _ = Image.open(path).convert('YCbCr').resize((40,40),resample=Image.BICUBIC).split()
        elif self.channel == 3:
            if self.resize: 
                upsample = Image.open(path).convert('RGB').resize((self.size,self.size),resample=Image.BICUBIC)
            else:
                upsample = Image.open(path).convert('RGB')
        else:
            raise Exception('Channel option not listed')
        
        data = (transforms.ToTensor()(upsample)).view(1, self.channel, upsample.size[1], upsample.size[0])
        data = data.to(device)
        out = self.model(data)
        out = out.cpu()
        out_img = out.data[0].numpy()
        out_img *= 255.0
        out_img = out_img.clip(0, 255)
                
        if self.channel == 1:
            out_img = Image.fromarray(np.uint8(out_img[0]), mode='L')
            out_img = Image.merge('YCbCr', [out_img, cb, cr]).convert('RGB')
            #out_img = out_img.resize((160,160), Image.BICUBIC)
        if self.channel == 3:
            out_img = out_img.transpose((1,2,0))
            out_img = Image.fromarray(np.uint8(out_img), mode='RGB')
            out_img = out_img.resize((160,160), Image.BICUBIC)

        return out_img
    
    
# Testing upsampling via Bicubic

def load_image(path='', size=160, resize=True): # Bicubic
    img = Image.open(path)
    if resize:
        img = img.resize((size,size), resample=Image.BICUBIC)
    return img

In [41]:
# SR Model Initialization

GPU_IN_USE = torch.cuda.is_available()

if GPU_IN_USE:
    cudnn.benchmark = True
    
device = torch.device('cuda' if GPU_IN_USE else 'cpu')
#model_path = '../checkpoints/SRCNN_coord_x4_epoch_50.pth'
#model_path = '../checkpoints/SRGAN_x4_epoch_30.pth'
#model_path = '../checkpoints/SRGAN_FaceLoss_x4_epoch_28.pth'
#model_path = '../checkpoints/SubCNN_coord_x4_epoch_50.pth'
#model_path = '../checkpoints/FSRCNN_coord_x4_epoch_50.pth'
#model_path = '../checkpoints/FSRCNN_coord_Loss_x4_epoch_29.pth'
model_path = '../checkpoints/SRGAN_coord_FaceLoss_x4_epoch_24.pth'


model = torch.load(model_path, map_location=lambda storage, loc: storage.cuda(0))
model = model.to(device)

In [42]:
resnet = InceptionResnetV1(pretrained='vggface2').eval()
#resnet = InceptionResnetV1(pretrained='casia-webface').eval()

In [43]:
def return_embedding(img_path):
    img_as_tensor = transforms.ToTensor()(load_image(img_path))
    img_emb = resnet(img_as_tensor.unsqueeze(0)).squeeze(0).detach().numpy()
    return img_emb

def return_batch_embedding(imgs_path):
    img_array = [transforms.ToTensor()(load_image(path)) for path in imgs_path]
    emb_array = [resnet(img.unsqueeze(0)).squeeze(0).detach().numpy() for img in img_array]
    return np.array(emb_array)

In [44]:
load_image = SuperResolveDL(model, size=56, channel=3, resize=True)

In [45]:
# Experimentation

name_of_classes = ['Classe3', 'Classe4', 'Classe5']

folder_data_size = ['NoSizeMargin/', '40/', '40-1.3/']

random.seed(1)
np.random.seed(1)

for name in name_of_classes:
    
    print('_____________________________________________________________')
    print(name)
    print('_____________________________________________________________')
    
    img_path = '/media/angelo/DATEN/Datasets/Dados_TCC_Joao/Material TCC/' + name + '/Classe/faces_bd/'

    for data_size in folder_data_size:
        print('For ' + data_size)

        gallery_path = gb.glob(img_path + 'gallery-' + data_size + '*.png')
        gallery_names = [os.path.splitext(name)[0] for name in os.listdir(img_path + 'gallery-' + data_size)]

        face_embeddings = return_batch_embedding(gallery_path)

        probe_path = img_path + 'probe-' + data_size
        class_folder = [folder for folder in os.listdir(probe_path) if os.path.isdir(os.path.join(probe_path, folder))] 
        average_accuracy = 0

        for classe in class_folder:

            accuracy = 0
            class_probe_path = probe_path + classe + '/'
            student_pool_list = os.listdir(class_probe_path)

            for index, student in enumerate(student_pool_list):

                student_emb = return_embedding(class_probe_path + student)

                similarities = [cos_sim(student_emb, face_embeddings[i]) for i in range(len(face_embeddings))]

                least_dis = np.argmin(np.array(similarities))

                if student[:2] == gallery_names[least_dis][:2]:
                    accuracy += 1

                #plt.figure(figsize=(5,5))
                #plt.subplot(1,2,1)
                #plt.imshow(load_image(gallery_path[least_dis]))
                #plt.subplot(1,2,2)
                #plt.imshow(load_image(class_probe_path + student))

            average_accuracy += accuracy/len(student_pool_list)

            print('Accuracy for Class ' + classe + ' was {:.4f}'.format(accuracy/len(student_pool_list)))

        print('Average accuracy was {:.2f}'.format(100*average_accuracy/3))

_____________________________________________________________
Classe3
_____________________________________________________________
For NoSizeMargin/
Accuracy for Class Classe_1 was 0.8125
Accuracy for Class Classe_2 was 0.8125
Accuracy for Class Classe_3 was 0.8125
Average accuracy was 81.25
For 40/
Accuracy for Class Classe_1 was 0.8125
Accuracy for Class Classe_2 was 0.8125
Accuracy for Class Classe_3 was 0.8125
Average accuracy was 81.25
For 40-1.3/
Accuracy for Class Classe_1 was 0.8125
Accuracy for Class Classe_2 was 0.8125
Accuracy for Class Classe_3 was 0.7500
Average accuracy was 79.17
_____________________________________________________________
Classe4
_____________________________________________________________
For NoSizeMargin/
Accuracy for Class Classe_1 was 0.5714
Accuracy for Class Classe_2 was 0.5000
Accuracy for Class Classe_3 was 0.5385
Average accuracy was 53.66
For 40/
Accuracy for Class Classe_1 was 0.5000
Accuracy for Class Classe_2 was 0.4286
Accuracy for Class